In [55]:
import flopy
import os
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

if sys.platform.lower()=='linux':
    datadir = Path('/scratch/users/ianpg/SWIlarge/data')
    workdir = Path('/scratch/users/ianpg/SWIlarge/work')
    MPSdir = datadir.joinpath('lith/sgems/MPS')
    lithdir = datadir.joinpath('lith/sgems/')
    GISdir = datadir.joinpath('GIS')
    priordir = datadir.joinpath('PriorModel')
    modeldir = datadir.joinpath('NM_model')
    import mplleaflet
elif sys.platform.lower()=='darwin':
    datadir = Path('../data')
    workdir = Path('../work')
    MPSdir = Path('/Users/ianpg/Dropbox/temp_convenience/SWIlarge/data/lith/sgems/MPS')
    GISdir = datadir.joinpath('GIS')
    lithdir = datadir.joinpath('lith/sgems/')
    priordir = datadir.joinpath('PriorModel')
    modeldir = datadir.joinpath('NM_model')

nmgwmdir_empty = datadir.joinpath('nmgwmdir_empty') #<-- removed everything but DIS
nmgwmdir_cal = datadir.joinpath('Calibrated_small') #<-- removed RCH, WEL, GLO, LST from the NAM file to load much faster
figdir = workdir.joinpath('figs')
outputdir = workdir.joinpath('output')

import config
from set_rc import *

import pandas as pd
import shapefile as sf #in case you dont have it, form anaconda prompt: pip install pyshp
from flopy.utils.gridgen import Gridgen
from flopy.utils.reference import SpatialReference
from  pyproj import Proj

p = Proj("epsg:26910")
xll=595855
yll = 4059438
rotation = -13.5


In [28]:
#%% Useful functions
def load_obj(dirname,name):
    import pickle
    with open(Path(dirname).joinpath(name + '.pkl').as_posix(), 'rb') as f:
        return pickle.load(f)

def save_obj(dirname,obj,name):
    import pickle
    with open(Path(dirname).joinpath(name + '.pkl').as_posix(), 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)




# INPUT

In [45]:
# it = int(sys.argv[1])
# f_varlist = Path(sys.argv[2])

it = 0
f_varlist = Path('../data/PriorModel/varlist.pkl')


### Load varlist

In [46]:
if len(f_varlist.name.split('.'))>1:
    nam = f_varlist.name.split('.')[0]
else:
    nam = f_varlist.name
varlist = load_obj(f_varlist.parent,nam)

In [47]:
varlist.keys()

dict_keys(['kvh', 'por_sand', 'por_clay', 'aL', 'TI_wavelength', 'TI_amplitude', 'TI_width', 'TI_direction', 'TI_prop', 'kh_sand_180', 'kh_clay_180', 'DSA_head', 'TSW_head', 'r_x', 'r_y', 'r_z', 'prop_400', 'sill', 'kh_sand_400', 'kh_clay_400', 'thinning', 'n_conduits', 'RP_model', 'm', 'CF_cp', 'CF_cr', 'seed', 'TI_props'])

### Load model

In [64]:
modelname = 'NM'
model_ws = workdir.joinpath("NM")
m= flopy.seawat.Seawat.load('NM.nam',exe_name=config.swexe, model_ws=model_ws.as_posix())
rows = np.load(model_ws.joinpath('rows.npy'))
layer_mapping_ind_full = np.load(GISdir.joinpath('layer_mapping_ind_full.npy'))                                 
layer_mapping_ind = layer_mapping_ind_full[:,rows,:]
# m = flopy.seawat.Seawat(modelname, exe_name=config.swexe, model_ws=model_ws.as_posix(),verbose=verbose)


### Set new values

#### Load new lith

In [68]:
[k for k,v in varlist.items()]

['kvh',
 'por_sand',
 'por_clay',
 'aL',
 'TI_wavelength',
 'TI_amplitude',
 'TI_width',
 'TI_direction',
 'TI_prop',
 'kh_sand_180',
 'kh_clay_180',
 'DSA_head',
 'TSW_head',
 'r_x',
 'r_y',
 'r_z',
 'prop_400',
 'sill',
 'kh_sand_400',
 'kh_clay_400',
 'thinning',
 'n_conduits',
 'RP_model',
 'm',
 'CF_cp',
 'CF_cr',
 'seed',
 'TI_props']

In [69]:
por_sand = varlist['por_sand'][it]
por_clay = varlist['por_clay'][it]
aL = varlist['aL'][it]
kvh = varlist['kvh'][it]
kh_sand_180 = varlist['kh_sand_180'][it]
kh_clay_180 = varlist['kh_clay_180'][it]
kh_sand_400 = varlist['kh_sand_400'][it]
kh_clay_400 = varlist['kh_clay_400'][it]

In [70]:
lith_180 = np.load(lithdir.joinpath('snesim','mps180_{}.npy'.format(it))).astype(np.float)
lith_400 = np.load(lithdir.joinpath('sisim','sisim400_{}.npy'.format(it))).astype(np.float)


lith_180[lith_180==1.] = kh_sand_180
lith_180[lith_180==0.] = kh_clay_180
lith_400[lith_400==1.] = kh_sand_400
lith_400[lith_400==0.] = kh_clay_400



In [42]:
hk = m.lpf.hk.array.copy()

mps180[mps180==1]=hkSand
mps180[mps180==0]=hkClay
hk[np.where(layer_mapping_ind==0)] = 10000
hk[np.where(layer_mapping_ind==1)] = 100
hk[np.where(layer_mapping_ind==2)] = hkClay*.01
hk[np.where(layer_mapping_ind==3)] = mps180[np.where(layer_mapping_ind==3)]
hk[np.where(layer_mapping_ind==4)] = hkClay*.01
hk[np.where(layer_mapping_ind==5)] = mps180[np.where(layer_mapping_ind==5)]


plt.figure()
plt.imshow(np.log10(hk)[:,0,:],vmin=-2,vmax=3)
plt.gca().set_aspect(10)
plt.colorbar()

array([[[625.  , 625.  , 625.  , ..., 336.6 , 336.6 , 336.6 ],
        [625.  , 625.  , 625.  , ..., 336.6 , 336.6 , 336.6 ]],

       [[625.  , 625.  , 625.  , ...,   5.  ,   5.  ,   5.  ],
        [625.  , 625.  , 625.  , ...,   5.  ,   5.  ,   5.  ]],

       [[625.  , 625.  , 625.  , ...,   5.  ,   5.  ,   5.  ],
        [625.  , 625.  , 625.  , ...,   5.  ,   5.  ,   5.  ]],

       ...,

       [[ 35.  ,  35.  ,  35.  , ...,   6.77,   6.77,   6.77],
        [ 35.  ,  35.  ,  35.  , ...,   6.77,   6.77,   6.77]],

       [[ 35.  ,  35.  ,  35.  , ...,  82.63,  82.53,  82.44],
        [ 35.  ,  35.  ,  35.  , ...,  82.73,  82.64,  82.55]],

       [[ 35.  ,  35.  ,  35.  , ...,  82.63,  82.53,  82.44],
        [ 35.  ,  35.  ,  35.  , ...,  82.73,  82.64,  82.55]]],
      dtype=float32)